In [1]:
# Load libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

In [2]:
# Read the dataset 
dataFile = "data/Shining32.csv"
data = pd.read_csv(dataFile)
data.head()

,buyerid,age,education,inc,gender,location,purchase,brandid,mint,white,fluoride,kids,size,discount,familypack,priceperoz,priceperpack,Chosen,SecondChoice
0,1,56,College,79523.203,Female,Brooklyn,Online,Colgate,Yes,Yes,No,No,4.8,1,0,0.92,4.42,0,NaN
1,1,56,College,79523.203,Female,Brooklyn,Online,Colgate,Yes,No,Yes,No,3.9,0,1,0.96,3.74,1,NaN
2,1,56,College,79523.203,Female,Brooklyn,Online,Colgate,No,Yes,Yes,Yes,3.9,1,0,1.04,4.06,0,NaN
3,1,56,College,79523.203,Female,Brooklyn,Online,Crest,Yes,Yes,No,No,4.8,1,0,0.92,4.42,0,NaN
4,1,56,College,79523.203,Female,Brooklyn,Online,Crest,No,No,Yes,Yes,4.8,1,0,1.01,4.85,0,NaN


In [83]:
# Processing

# Convert yes/no to 1/0
dict  = {"Yes" : 1, "No" : 0}
data.replace({"mint": dict,"white": dict,"fluoride": dict, "kids": dict},inplace=True)

# Define product id
uniqueProducts  = data.groupby(['brandid','mint','white','fluoride','kids']).size().reset_index().rename(columns={0:'count'})
data["productId"] = 0
for i in range(10):
    data.iloc[(data.brandid == uniqueProducts.iloc[i,0]) & 
        (data.mint == uniqueProducts.iloc[i,1]) &
        (data.white == uniqueProducts.iloc[i,2]) &
        (data.fluoride == uniqueProducts.iloc[i,3]) &
        (data.kids == uniqueProducts.iloc[i,4]),-1] = i + 1

# Add dummies
data = data.join(pd.get_dummies(data.education,prefix="ed"))
data = data.join(pd.get_dummies(data.location,prefix="loc"))
data = data.join(pd.get_dummies(data.gender,prefix="gen"))
data = data.join(pd.get_dummies(data.purchase,prefix="purchase"))
data = data.join(pd.get_dummies(data.brandid,prefix="brand"))

# Rename some columns
data = data.rename({'ed_High School':'ed_HighSchool', 'ed_More than College':'ed_MoreCollege',
     'purchase_In Store':'purchase_InStore'}, axis='columns')

data.head(12)

,buyerid,age,education,inc,gender,location,purchase,brandid,mint,white,...,loc_Queens,gen_Female,gen_Male,gen_Other,purchase_InStore,purchase_Online,brand_Aquafresh,brand_Colgate,brand_Crest,brand_Sensodyne
0,1,56,College,79523.203,Female,Brooklyn,Online,Colgate,1,1,...,0,1,0,0,0,1,0,1,0,0
1,1,56,College,79523.203,Female,Brooklyn,Online,Colgate,1,0,...,0,1,0,0,0,1,0,1,0,0
2,1,56,College,79523.203,Female,Brooklyn,Online,Colgate,0,1,...,0,1,0,0,0,1,0,1,0,0
3,1,56,College,79523.203,Female,Brooklyn,Online,Crest,1,1,...,0,1,0,0,0,1,0,0,1,0
4,1,56,College,79523.203,Female,Brooklyn,Online,Crest,0,0,...,0,1,0,0,0,1,0,0,1,0
5,1,56,College,79523.203,Female,Brooklyn,Online,Sensodyne,1,0,...,0,1,0,0,0,1,0,0,0,1
6,1,56,College,79523.203,Female,Brooklyn,Online,Sensodyne,0,1,...,0,1,0,0,0,1,0,0,0,1
7,1,56,College,79523.203,Female,Brooklyn,Online,Aquafresh,1,0,...,0,1,0,0,0,1,1,0,0,0
8,1,56,College,79523.203,Female,Brooklyn,Online,Aquafresh,0,1,...,0,1,0,0,0,1,1,0,0,0
9,1,56,College,79523.203,Female,Brooklyn,Online,Aquafresh,0,1,...,0,1,0,0,0,1,1,0,0,0


In [84]:
dictBorough = {"Brooklyn": 1, "Manhattan": 2, "Queens": 3, "Other":4}
data["market"] = data["location"].map(dictBorough)
data.head(20)

,buyerid,age,education,inc,gender,location,purchase,brandid,mint,white,...,gen_Female,gen_Male,gen_Other,purchase_InStore,purchase_Online,brand_Aquafresh,brand_Colgate,brand_Crest,brand_Sensodyne,market
0,1,56,College,79523.203,Female,Brooklyn,Online,Colgate,1,1,...,1,0,0,0,1,0,1,0,0,1
1,1,56,College,79523.203,Female,Brooklyn,Online,Colgate,1,0,...,1,0,0,0,1,0,1,0,0,1
2,1,56,College,79523.203,Female,Brooklyn,Online,Colgate,0,1,...,1,0,0,0,1,0,1,0,0,1
3,1,56,College,79523.203,Female,Brooklyn,Online,Crest,1,1,...,1,0,0,0,1,0,0,1,0,1
4,1,56,College,79523.203,Female,Brooklyn,Online,Crest,0,0,...,1,0,0,0,1,0,0,1,0,1
5,1,56,College,79523.203,Female,Brooklyn,Online,Sensodyne,1,0,...,1,0,0,0,1,0,0,0,1,1
6,1,56,College,79523.203,Female,Brooklyn,Online,Sensodyne,0,1,...,1,0,0,0,1,0,0,0,1,1
7,1,56,College,79523.203,Female,Brooklyn,Online,Aquafresh,1,0,...,1,0,0,0,1,1,0,0,0,1
8,1,56,College,79523.203,Female,Brooklyn,Online,Aquafresh,0,1,...,1,0,0,0,1,1,0,0,0,1
9,1,56,College,79523.203,Female,Brooklyn,Online,Aquafresh,0,1,...,1,0,0,0,1,1,0,0,0,1


In [68]:
# Fit some logit models to explain chosen product characteristcs

# Restrict the dataset to the chosen prodicts
dataChosen = data[data['Chosen'] == 1]

# Normalization
dataChosen["sizeNorm"] = np.divide(dataChosen["size"]-np.min(dataChosen["size"]),
                                np.max(dataChosen["size"])-np.min(dataChosen["size"]))
dataChosen["inc"] = dataChosen["inc"]/1000

# Regressors = consumer attributes
regressors  = '~ age + inc + ed_College + ed_MoreCollege +'\
                'loc_Brooklyn + loc_Manhattan + loc_Queens +'\
                'gen_Female + gen_Male + purchase_InStore'

# Predict the chosen brand
mBrand_1 = smf.logit('brand_Colgate'+regressors, data=dataChosen).fit()
mBrand_2 = smf.logit('brand_Aquafresh'+regressors, data=dataChosen).fit()
mBrand_3 = smf.logit('brand_Sensodyne'+regressors, data=dataChosen).fit()
mBrand_4 = smf.logit('brand_Crest'+regressors, data=dataChosen).fit()

# Predict other chosen characteristics
mChrc_1 = smf.logit('mint'+regressors, data=dataChosen).fit()
mChrc_2 = smf.logit('white'+regressors, data=dataChosen).fit()
mChrc_3 = smf.logit('fluoride'+regressors, data=dataChosen).fit()
mChrc_4 = smf.logit('kids'+regressors, data=dataChosen).fit()
mChrc_5 = smf.logit('sizeNorm'+regressors, data=dataChosen).fit()
mChrc_6 = smf.logit('discount'+regressors, data=dataChosen).fit()
mChrc_7 = smf.logit('familypack'+regressors, data=dataChosen).fit()

C:\Users\Rob\AppData\Local\Temp\ipykernel_43480\2617437204.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataChosen["sizeNorm"] = np.divide(dataChosen["size"]-np.min(dataChosen["size"]),
C:\Users\Rob\AppData\Local\Temp\ipykernel_43480\2617437204.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataChosen["inc"] = dataChosen["inc"]/1000


Optimization terminated successfully.
         Current function value: 0.598719
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.377925
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.313782
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.237058
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.154480
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.537584
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.618602
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.079393
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.565306
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.342223
  

In [72]:
resultsBrand = summary_col([mBrand_1,mBrand_2,mBrand_3,mBrand_4],stars=True)
print(resultsBrand)


                 brand_Colgate brand_Aquafresh brand_Sensodyne brand_Crest
--------------------------------------------------------------------------
Intercept        -0.2597       -1.5713**       -1.0961         -1.4053    
                 (0.4718)      (0.6451)        (0.7031)        (0.8821)   
age              0.0093        -0.0099         -0.0068         -0.0037    
                 (0.0059)      (0.0080)        (0.0091)        (0.0109)   
inc              0.0041        -0.0022         0.0027          -0.0138*   
                 (0.0044)      (0.0060)        (0.0068)        (0.0082)   
ed_College       -0.0211       0.4424***       -0.2242         -0.1874    
                 (0.1047)      (0.1656)        (0.1497)        (0.1744)   
ed_MoreCollege   -0.0575       0.8020***       -0.6473***      -0.2464    
                 (0.1362)      (0.2016)        (0.2057)        (0.2401)   
loc_Brooklyn     0.2505        -0.0837         -0.2198         -0.3498    
                 (0.2544

In [70]:
resultsChrc = summary_col([mChrc_1,mChrc_2,mChrc_3,mChrc_4,mChrc_5,mChrc_6,mChrc_7],stars=True)
print(resultsChrc)


                    mint      white     fluoride    kids    sizeNorm   discount  familypack
-------------------------------------------------------------------------------------------
Intercept        3.1499***  -0.1860    -0.6667   -2.3758   -1.7106*** 3.3196***  0.4736    
                 (1.0684)   (0.5300)   (0.4709)  (1.6867)  (0.4962)   (0.6971)   (0.4414)  
age              0.0041     -0.0022    0.0042    -0.0077   0.0034     0.0013     -0.0149***
                 (0.0140)   (0.0063)   (0.0058)  (0.0214)  (0.0061)   (0.0086)   (0.0054)  
inc              0.0095     -0.0098**  0.0079*   -0.0271*  0.0040     -0.0177*** 0.0032    
                 (0.0105)   (0.0048)   (0.0043)  (0.0161)  (0.0046)   (0.0065)   (0.0041)  
ed_College       -0.2839    -0.5879*** 0.6102*** 0.1196    0.3051**   -0.0424    -0.1177   
                 (0.2613)   (0.1009)   (0.0967)  (0.3608)  (0.1187)   (0.1652)   (0.0959)  
ed_MoreCollege   -0.3866    -1.1449*** 1.1959*** 0.4891    0.6475***  -0.0737  

In [86]:
data.columns

Index(['buyerid', 'age', 'education', 'inc', 'gender', 'location', 'purchase',
       'brandid', 'mint', 'white', 'fluoride', 'kids', 'size', 'discount',
       'familypack', 'priceperoz', 'priceperpack', 'Chosen', 'SecondChoice',
       'productId', 'ed_College', 'ed_HighSchool', 'ed_MoreCollege',
       'loc_Brooklyn', 'loc_Manhattan', 'loc_Other', 'loc_Queens',
       'gen_Female', 'gen_Male', 'gen_Other', 'purchase_InStore',
       'purchase_Online', 'brand_Aquafresh', 'brand_Colgate', 'brand_Crest',
       'brand_Sensodyne', 'market'],
      dtype='object')

In [96]:
vars = ["mint","white","fluoride","kids","priceperpack","productId"]
datamkt = data[vars].groupby(["productId"]).agg("mean")
datamkt.head()

,mint,white,fluoride,kids,priceperpack
productId,,,,,
1,0.0,1.0,0.0,0.0,4.200341
2,0.0,1.0,0.0,1.0,4.626885
3,1.0,0.0,1.0,0.0,4.144241
4,0.0,1.0,1.0,1.0,4.732205
5,1.0,0.0,1.0,0.0,4.147953


In [98]:
sales  = data[["Chosen","productId"]].groupby(["productId"]).agg("sum")
datamkt = datamkt.join(sales)
datamkt.head()

,mint,white,fluoride,kids,priceperpack,Chosen
productId,,,,,,
1,0.0,1.0,0.0,0.0,4.200341,208
2,0.0,1.0,0.0,1.0,4.626885,43
3,1.0,0.0,1.0,0.0,4.144241,1018
4,0.0,1.0,1.0,1.0,4.732205,41
5,1.0,0.0,1.0,0.0,4.147953,5597


In [74]:
resultsBrand.as_latex()

'\\begin{table}\n\\caption{}\n\\label{}\n\\begin{center}\n\\begin{tabular}{lllll}\n\\hline\n                  & brand\\_Colgate & brand\\_Aquafresh & brand\\_Sensodyne & brand\\_Crest  \\\\\n\\hline\nIntercept         & -0.2597        & -1.5713**        & -1.0961          & -1.4053       \\\\\n                  & (0.4718)       & (0.6451)         & (0.7031)         & (0.8821)      \\\\\nage               & 0.0093         & -0.0099          & -0.0068          & -0.0037       \\\\\n                  & (0.0059)       & (0.0080)         & (0.0091)         & (0.0109)      \\\\\ninc               & 0.0041         & -0.0022          & 0.0027           & -0.0138*      \\\\\n                  & (0.0044)       & (0.0060)         & (0.0068)         & (0.0082)      \\\\\ned\\_College       & -0.0211        & 0.4424***        & -0.2242          & -0.1874       \\\\\n                  & (0.1047)       & (0.1656)         & (0.1497)         & (0.1744)      \\\\\ned\\_MoreCollege   & -0.0575        & 0

In [76]:
resultsChrc.as_latex()

'\\begin{table}\n\\caption{}\n\\label{}\n\\begin{center}\n\\begin{tabular}{llllllll}\n\\hline\n                  & mint       & white      & fluoride  & kids      & sizeNorm   & discount   & familypack  \\\\\n\\hline\nIntercept         & 3.1499***  & -0.1860    & -0.6667   & -2.3758   & -1.7106*** & 3.3196***  & 0.4736      \\\\\n                  & (1.0684)   & (0.5300)   & (0.4709)  & (1.6867)  & (0.4962)   & (0.6971)   & (0.4414)    \\\\\nage               & 0.0041     & -0.0022    & 0.0042    & -0.0077   & 0.0034     & 0.0013     & -0.0149***  \\\\\n                  & (0.0140)   & (0.0063)   & (0.0058)  & (0.0214)  & (0.0061)   & (0.0086)   & (0.0054)    \\\\\ninc               & 0.0095     & -0.0098**  & 0.0079*   & -0.0271*  & 0.0040     & -0.0177*** & 0.0032      \\\\\n                  & (0.0105)   & (0.0048)   & (0.0043)  & (0.0161)  & (0.0046)   & (0.0065)   & (0.0041)    \\\\\ned\\_College       & -0.2839    & -0.5879*** & 0.6102*** & 0.1196    & 0.3051**   & -0.0424    & -